In [ ]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup

In [ ]:
def get_moves(dexno, df):
  def add_move(df, method, name, link):
      if len(df) == 0:
        index = 0
      else:
        index = df.index.max() + 1
      df.loc[index] = [dexno, name, link, method]


  res = requests.get(f"https://www.serebii.net/pokedex-bw/{dexno:03d}.shtml")
  soup = BeautifulSoup(res.text, 'html.parser')
  for table in soup.findAll('table', class_='dextable'):
    table_name = table.find("td", class_="fooevo")
    if table_name:
      if table_name.text.endswith("Level Up"):
        i = -1
        for row in table.findAll('tr'):
          i += 1
          if i % 2 == 1 or i == 0:
            continue
          if row.find('td').text == "—":
            level = 0
          else:
            level = int(row.find('td').text)
          method = f"Level {level}"
          name = row.find('a').text
          link = f"https://www.serebii.net/{row.find('a').get('href')}"
          add_move(df, method, name, link)
      elif "TM" in table_name.text:
        i = -1
        for row in table.findAll('tr'):
          i += 1
          if i % 3 != 2 or i == 0:
            continue
          method = row.find('td').text
          name = row.find('a').text
          link = f"https://www.serebii.net/{row.find('a').get('href')}"
          add_move(df, method, name, link)
      elif "Tutor" in table_name.text:
        #continue
        method = "Move Tutor"
        i = -1
        for row in table.findAll('tr'):
          i += 1
          if i % 3 != 2 or i == 0:
            continue
          name = row.find('a').text
          link = f"https://www.serebii.net/{row.find('a').get('href')}"
          add_move(df, method, name, link)
      elif "Egg" in table_name.text:
        #continue
        method = "Breeding"
        i = -1
        for row in table.findAll('tr'):
          i += 1
          if i % 3 != 2 or i == 0:
            continue
          name = row.find('a').text
          link = f"https://www.serebii.net/{row.find('a').get('href')}"
          add_move(df, method, name, link)
      elif "Pre-Evolution" in table_name.text:
        #continue
        method = "Pre-evolution"
        i = -1
        for row in table.findAll('tr'):
          i += 1
          if i % 2 == 1 or i == 0:
            continue
          name = row.find('a').text
          link = f"https://www.serebii.net/{row.find('a').get('href')}"
          add_move(df, method, name, link)
      elif "III" in table_name.text or "IV" in table_name.text:
        #continue
        i = -1
        if len(table.findAll('tr')[2].findAll('th')) == 3:
          i -= 1
        for row in table.findAll('tr'):
          i += 1
          if i % 2 == 1 or i == 0:
            continue
          method = row.findAll('td')[-1].text
          name = row.find('a').text
          link = f"https://www.serebii.net/{row.find('a').get('href')}"
          add_move(df, method, name, link)







In [ ]:
df3 = pd.DataFrame(columns=['pokemon_id', 'move_name', 'move_url', 'method'])

In [ ]:
get_moves(413, df3)

In [ ]:
df3.to_csv("form_difference_moves.csv")

In [ ]:
for i in [487, 492, 641, 642, 645, 646, 647]:
  print(i, end=", ")
  get_moves(i, df3)

487, 492, 641, 642, 645, 646, 647, 

In [ ]:
df3 = df3[df3['pokemon_id'] < 641]

In [ ]:
df3[df3["move_name"] == "Psybeam"]

,pokemon_id,move_name,move_url,method
812,12,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 24
3565,46,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Breeding
3644,47,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Breeding
3679,48,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 25
3740,49,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 25
...,...,...,...,...
45977,575,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 16
46057,576,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 16
47770,605,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 15
47850,606,Psybeam,https://www.serebii.net//attackdex-bw/psybeam....,Level 15


In [ ]:
df3.to_csv("moves_to_pokemon.csv")

In [ ]:
def get_pokemon_details(df, page):
  res = requests.get(page)
  soup = BeautifulSoup(res.text, 'html.parser')
  name = soup.findAll('td', class_='fooinfo')[0].text
  types = soup.findAll('table', class_='dextable')[1].findAll('td', class_='fooinfo')[4].findAll('a')
  type1 = types[0].get('href')[12:-6]
  if len(types) == 2:
    type2 = types[1].get('href')[12:-6]
  else:
    type2 = None
  abilities = soup.findAll('table', class_='dextable')[1].findAll('td', class_='fooinfo')[5].findAll('a')
  ability1 = abilities[0].text
  ability2 = abilities[1].text if len(abilities) == 3 else None
  hidden_ability = abilities[-1].text
  stats = soup.findAll('table', class_='dextable')[-1].findAll('td', class_='fooinfo')
  hp = int(stats[1].text)
  attack = int(stats[2].text)
  defense = int(stats[3].text)
  special_attack = int(stats[4].text)
  special_defense = int(stats[5].text)
  speed = int(stats[6].text)
  total = hp + attack + defense + special_attack + special_defense + speed
  print(name, end=", ")
  if len(df) == 0:
    index = 0
  else:
    index = df.index.max() + 1
  df.loc[index] = [page, name, type1, type2, ability1, ability2, hidden_ability, hp, attack, defense, special_attack, special_defense, speed, total]


In [ ]:
df2 = pd.DataFrame(columns = ['url', 'name', 'type1', 'type2', 'ability1', 'ability2', 'hidden_ability', 'hp', 'attack', 'defense', 'special_attack', 'special_defense', 'speed', 'total'])

In [ ]:
for i in range(1, 650):
  get_pokemon_details(df2, f"https://www.serebii.net/pokedex-bw/{i:03d}.shtml")
  if i % 30 == 0:
    print()

Bulbasaur, Ivysaur, Venusaur, Charmander, Charmeleon, Charizard, Squirtle, Wartortle, Blastoise, Caterpie, Metapod, Butterfree, Weedle, Kakuna, Beedrill, Pidgey, Pidgeotto, Pidgeot, Rattata, Raticate, Spearow, Fearow, Ekans, Arbok, Pikachu, Raichu, Sandshrew, Sandslash, Nidoran♀, Nidorina, 
Nidoqueen, Nidoran♂, Nidorino, Nidoking, Clefairy, Clefable, Vulpix, Ninetales, Jigglypuff, Wigglytuff, Zubat, Golbat, Oddish, Gloom, Vileplume, Paras, Parasect, Venonat, Venomoth, Diglett, Dugtrio, Meowth, Persian, Psyduck, Golduck, Mankey, Primeape, Growlithe, Arcanine, Poliwag, 
Poliwhirl, Poliwrath, Abra, Kadabra, Alakazam, Machop, Machoke, Machamp, Bellsprout, Weepinbell, Victreebel, Tentacool, Tentacruel, Geodude, Graveler, Golem, Ponyta, Rapidash, Slowpoke, Slowbro, Magnemite, Magneton, Farfetch'd, Doduo, Dodrio, Seel, Dewgong, Grimer, Muk, Shellder, 
Cloyster, Gastly, Haunter, Gengar, Onix, Drowzee, Hypno, Krabby, Kingler, Voltorb, Electrode, Exeggcute, Exeggutor, Cubone, Marowak, Hitmonlee,

In [ ]:
df2.to_csv("pokemon_species.csv")

In [ ]:
res = requests.get("https://www.serebii.net/attackdex-bw/")
soup = BeautifulSoup(res.text, 'html.parser')

move_pages = []
for i in range(3):
  for option in soup.findAll('select')[i].findAll('option'):
    if option.get('value'):
      move_pages.append('https://www.serebii.net' + option.get('value'))

In [ ]:
def get_move_details(df, move_page):
  res = requests.get(move_page)
  soup = BeautifulSoup(res.text, 'html.parser')
  table = soup.find('table', class_='dextable').findAll('tr')
  move_name = re.search(r'[a-zA-Z \'\-0-9]+', table[1].td.text.strip()).group().strip()
  print(move_name, end=", ")
  move_type = re.search(r'/(\w+).shtml', table[1].findAll('td')[1].a.get('href')).group(1)
  move_category = re.search(r'/(\w+).shtml', table[1].findAll('td')[2].a.get('href')).group(1)
  move_pp = int(table[3].findAll('td')[0].text.strip())
  move_power = int(table[3].findAll('td')[1].text.strip())
  move_accuracy = int(table[3].findAll('td')[2].text.strip())
  move_description = table[5].td.text.strip()
  move_secondary_effect = table[7].td.text.strip()
  if len(df) == 0:
    index = 0
  else:
    index = df.index.max() + 1
  df.loc[index] = [move_page, move_name, move_type, move_category, move_pp, move_power, move_accuracy, move_description, move_secondary_effect]



In [ ]:
df = pd.DataFrame(columns = ['url', 'move_name', 'move_type', 'move_category', 'move_pp', 'move_power', 'move_accuracy', 'move_description', 'move_secondary_effect'])

In [ ]:
count = 0
for page in move_pages:
  count += 1
  get_move_details(df, page)
  if count % 20 == 0:
    print()

Absorb, Acid, Acid Armor, Acid Spray, Acrobatics, Acupressure, Aerial Ace, Aeroblast, After You, Agility, Air Cutter, Air Slash, Ally Switch, Amnesia, Ancientpower, Aqua Jet, Aqua Ring, Aqua Tail, Arm Thrust, Aromatherapy, 
Assist, Assurance, Astonish, Attack Order, Attract, Aura Sphere, Aurora Beam, Autotomize, Avalanche, Barrage, Barrier, Baton Pass, Beat Up, Belly Drum, Bestow, Bide, Bind, Bite, Blast Burn, Blaze Kick, 
Blizzard, Block, Blue Flare, Body Slam, Bolt Strike, Bone Club, Bone Rush, Bonemerang, Bounce, Brave Bird, Brick Break, Brine, Bubble, Bubblebeam, Bug Bite, Bug Buzz, Bulk Up, Bulldoze, Bullet Punch, Bullet Seed, 
Calm Mind, Camouflage, Captivate, Charge, Charge Beam, Charm, Chatter, Chip Away, Circle Throw, Clamp, Clear Smog, Close Combat, Coil, Comet Punch, Confuse Ray, Confusion, Constrict, Conversion, Conversion 2, Copycat, 
Cosmic Power, Cotton Guard, Cotton Spore, Counter, Covet, Crabhammer, Cross Chop, Cross Poison, Crunch, Crush Claw, Crush Grip, Curse, Cut, 

In [ ]:
df.head()

,url,move_name,move_type,move_category,move_pp,move_power,move_accuracy,move_description,move_secondary_effect
0,https://www.serebii.net/attackdex-bw/absorb.shtml,Absorb,grass,special,25,20,100,A nutrient-draining attack. The user's HP is r...,User recovers half the damage inflicted.
1,https://www.serebii.net/attackdex-bw/acid.shtml,Acid,poison,special,30,40,100,The opposing team is attacked with a spray of ...,May lower opponent's SP. DEF one stage.
2,https://www.serebii.net/attackdex-bw/acidarmor...,Acid Armor,poison,other,40,0,0,The user alters its cellular structure to liqu...,Raises user's DEFENSE two stages.
3,https://www.serebii.net/attackdex-bw/acidspray...,Acid Spray,poison,special,20,40,100,The user spits fluid that works to melt the ta...,Lowers opponent's SP. DEF two stages.
4,https://www.serebii.net/attackdex-bw/acrobatic...,Acrobatics,flying,physical,15,55,100,The user nimbly strikes the target. If the use...,Increases damage by 100% if user is not holdin...


In [ ]:
df.to_csv("pokemon_moves.csv")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
